# 용산구 (1100)
- 작성자 : 황다은
- 작성일자 : 2020년 9월 9일
- 비고에 마스크 착용여부, 이동수단 있으나 전처리 하지 못했음.

In [109]:
from bs4 import BeautifulSoup
import datetime as dt
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

dat = dt.datetime.today().date() # 오늘 날짜

### 1. Raw File 불러오기

In [110]:
df = pd.read_csv("200908one_shot.csv")
soup = BeautifulSoup(df['html'][4], 'html.parser')

### 2. 간단한 전처리 하기

In [111]:
rows = soup.select('.coronaV-state-table')
tablelist = rows[0].select('.sub_table')
len_of_list = len(tablelist)

### 3. 저장할 DataFrame 미리 만들기

In [116]:
pat_col = ['num', 'dat', 'pth', 'syy', 'syd', 'cnd', 'rsd', 'atn', 'dch', 'adt','age','sex']
rou_col = ['num', 'ord', 'dat', 'rgl', 'rgt', 'frm', 'exd', 'msk', 'ste', 'mob']
patient = pd.DataFrame(columns = pat_col)
route = pd.DataFrame(columns = rou_col)

### 4. 귀찮은 전처리해주기

In [119]:
for i in tqdm(range(len(rows[0].select('.corona_info')))):
    #patient는 확진자 1번부터 마지막확진자까지 데이터 전부 있음.
    title = rows[0].select('.corona_info')[i].select('td')
    numindex = title[0].text.index('(')
    
    num = '1100' + '0' * (5- len(title[0].text[3:numindex])) + title[0].text[3:numindex]
    dch = title[1].text
    cnd = title[2].text
    pth = title[3].text
    atn = title[4].text
    if 2*i<len_of_list:        
        subtable = pd.read_html(str(tablelist[2*i+1]))[0]
        if '명' in str(subtable[1]):
            adtindex = str(subtable[1]).index('명')
            adt = str(subtable[1])[adtindex-2:adtindex]
        else:
            adt = 0
    else:
        adt = ''
    syy = ''; syd = ''; rsd = '' ; sex = ''; age = ''
    new_pat = pd.DataFrame([(num, dat, pth, syy, syd, cnd, rsd, atn, dch, adt, age, sex)], columns = pat_col)
    patient = pd.concat([patient, new_pat])
    
    #route는 if문에 해당될 경우만, 데이터 존재.
    if 2*i<len_of_list:  
        new_route = pd.read_html(str(tablelist[2*i]))[0]
        new_route = new_route.drop(['비고'],axis=1)
        new_route['num'] = num
        new_route['ord'] = range(1, len(new_route)+1)
        new_route['dat'] = dat
        new_route['msk'] = ''; new_route['mob']=''  #msk와 mob는 비고에 있음.
        new_route.columns = ['rgl', 'frm', 'rgt', 'exd', 'ste', 'num', 'ord', 'dat', 'msk', 'mob']

    #route 데이터 사라졌을 시 빈행추가 
    else:
        new_route = pd.DataFrame(columns = rou_col, data = [[num, 1, dat, '', '', '', '', '', '', '']])

    route = pd.concat([route, new_route])

100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [00:01<00:00, 65.63it/s]


### 5. 결과 잘 나왔나 확인하기

In [114]:
patient

,num,dat,pth,syy,syd,cnd,rsd,atn,dch,adt,age,sex
0,11000108,2020-09-09,타구 소재 종교시설 집회 참석,,,9.7.(월) 12:00,,북부병원(9.7. 17:10),-,1,,
0,11000107,2020-09-09,타지역(동대문구) 확진자 접촉,,,9.4.(금) 09:00,,적십자병원(9.4. 14:00),-,0,,
0,11000106,2020-09-09,용산#105 확진자(가족) 접촉,,,9.4.(금) 09:00,,서울의료원(9.4. 14:10),-,0,,
0,11000105,2020-09-09,확인 중,,,9.3.(목) 09:00,,서울의료원(9.3. 14:00),-,2,,
0,11000104,2020-09-09,확인 중,,,9.3.(목) 09:00,,코이카생활치료센터(9.3. 16:50),-,6,,
...,...,...,...,...,...,...,...,...,...,...,...,...
0,11000005,2020-09-09,용산#4(#8486) 접촉,,,3.19.(목) 09:00,,흑석동 중앙대병원 (3.19. 16:00),완치,,,
0,11000004,2020-09-09,구로구 콜센터 관련,,,3.18.(수) 09:20,,흑석동 중앙대병원 (3.18. 16:00),완치,,,
0,11000003,2020-09-09,타지역 확진자(#7265) 접촉 (구로구 콜센터 관련),,,3.16.(월) 11:00,,상계백병원 (3.16. 19:00),완치,,,
0,11000002,2020-09-09,해외 감염,,,3.12.(목) 13:00,,흑석동 중앙대병원 (3.12. 21:00),완치,,,


In [118]:
route

,num,ord,dat,rgl,rgt,frm,exd,msk,ste,mob
0,11000108,1,2020-09-09,타구,타구 동선,종교시설,8.25.(화) 06:40~11:30 8.26.(수) 06:40~09:30 8....,,NaN,
1,11000108,2,2020-09-09,용산구,자택,거주지,8.30.(일),,방역 완료,
2,11000108,3,2020-09-09,후암동,미공개,병원,8.27.(목) 11:22 8.31.(월) 14:37,,방역 완료,
3,11000108,4,2020-09-09,후암동,미공개,약국,8.27.(목) 11:37 8.31.(월) 14:47,,방역 완료,
4,11000108,5,2020-09-09,타구,보건소 선별진료소 검체채취,공공시설,9.6.(일) 12:30~16:00,,방역 완료,
...,...,...,...,...,...,...,...,...,...,...
0,11000005,1,2020-09-09,,,,,,,
0,11000004,1,2020-09-09,,,,,,,
0,11000003,1,2020-09-09,,,,,,,
0,11000002,1,2020-09-09,,,,,,,
